## Installing Packages

In [13]:
from hashlib import sha1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

plt.rcParams["font.size"] = 16

from sklearn.compose import (
    ColumnTransformer,
    TransformedTargetRegressor,
    make_column_transformer,
)
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, train_test_split 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,KBinsDiscretizer
from lightgbm.sklearn import LGBMRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
# import shap

ImportError: cannot import name 'SimpleImputer' from 'sklearn.preprocessing' (/Users/vivaanwadhwa/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/preprocessing/__init__.py)

## Reading Dataset

In [2]:
df = pd.read_csv("data/data.csv")

In [3]:
train_df, test_df= train_test_split(df,test_size=0.3, random_state=123)
train_df, test_df= train_df.reset_index(), test_df.reset_index()

In [4]:
train_df.head()

,index,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,592965,for_sale,4.0,2.0,1.69,Lebanon,New Jersey,8826.0,NaN,2014-05-22,389900.0
1,132488,for_sale,2.0,2.0,0.16,Belmont,Massachusetts,2478.0,2452.0,2003-09-22,1450000.0
2,235274,for_sale,2.0,1.0,0.09,Providence,Rhode Island,2903.0,1395.0,NaN,309000.0
3,283435,for_sale,8.0,4.0,0.14,Haverhill,Massachusetts,1830.0,2418.0,NaN,599000.0
4,813679,for_sale,3.0,1.0,0.07,Mount Vernon,New York,10550.0,1700.0,2009-09-03,200000.0


## Preliminary Analysis

In [5]:
train_df.shape

(633476, 11)

In [6]:
train_df.describe()

,index,bed,bath,acre_lot,zip_code,house_size,price
count,633476.000000,542599.000000,553794.000000,446808.000000,633329.000000,4.285600e+05,6.334270e+05
mean,452091.471334,3.331930,2.485547,16.944473,6513.814735,2.140715e+03,8.809529e+05
std,261284.123710,2.064298,1.898663,948.028332,3818.076267,3.249101e+03,2.676120e+06
min,0.000000,1.000000,1.000000,0.000000,601.000000,1.000000e+02,0.000000e+00
25%,225849.500000,2.000000,2.000000,0.110000,2908.000000,1.134000e+03,2.690000e+05
50%,451798.500000,3.000000,2.000000,0.290000,6798.000000,1.654000e+03,4.750000e+05
75%,678469.250000,4.000000,3.000000,1.150000,8853.000000,2.499000e+03,8.315000e+05
max,904963.000000,123.000000,198.000000,100000.000000,99999.000000,1.450112e+06,8.750000e+08


In [7]:
train_df.isnull().sum()

index                  0
status                 0
bed                90877
bath               79682
acre_lot          186668
city                  53
state                  0
zip_code             147
house_size        204916
prev_sold_date    321701
price                 49
dtype: int64

### Cleaning the Data

In [16]:
train_df['NewHouse'] = train_df['prev_sold_date'].isnull().apply(lambda x: 1 if x else 0)

In [14]:
X_train = train_df.drop(columns=["price"])
y_train = train_df["price"]

X_test = test_df.drop(columns=["price"])
y_test = test_df["price"]